<a href="https://colab.research.google.com/github/tbradshaw91/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things
1 + 4

5

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [3]:

%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import metrics
!pip install ggplot
from ggplot import *
import matplotlib.pyplot as plt
import seaborn as sns


import pandas as pd
import urllib.request as request
import csv


# For some reason, this is where it breaks when I run it on my own. I keep getting a 404 error
data = pd.read_csv('https://raw.githubusercontent.com/JoomiK/WineDrinkers/datasets/master/WineDrinkers.csv', );

# Look at the data
data.describe()

# The most popular data
data.mean().plot(kind='bar')

HTTPError: ignored

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




1. For this assignment, I searched Github and found a Wine dataset, I pulled it in but kept getting an error.. Not sure if the link is broken or what is going on. 
2. The most challenging part was reading in the dataset, I kept running into error messages. I will spend some time this evening after class working on this. 
3. The most interesting thing was seeing the dataset represented in many different ways. That is what drew me to this example.
4. With more time, I would like to solve the error with the dataset and get it working 100%

## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.